# Bushel Management Reports

This notebook provides read-only access to the Bushel Management database for reporting and analysis.

## Setup

First, mount Google Drive and set up the database connection.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Install required packages
!pip install sqlalchemy pandas matplotlib seaborn python-dateutil


In [ ]:
# Set database path (update this to match your Google Drive folder structure)
DB_PATH = '/content/drive/MyDrive/Colab_Notebooks/Grain_Manager/database/bushel_management.db'

# Alternative: If database is in a different location, update the path above
# Example: '/content/drive/MyDrive/YourFolder/bushel_management.db'

# Set project path (update this to match your Google Drive folder name)
# This should be the folder where you uploaded the Bushel_Management_Reports project
PROJECT_PATH = '/content/drive/MyDrive/Colab_Notebooks/Grain_Manager'

# Alternative: If you uploaded to a different folder name, update PROJECT_PATH above
# For example: '/content/drive/MyDrive/Bushel_Management_Reports'


## 🚀 Standalone Dashboard Application

**Run the dashboard as a standalone web application!**

This launches the Enhanced Bushel Management Dashboard in a separate web interface that feels like a native application. The dashboard will open in a new window/tab.


In [ ]:
# Standalone Dashboard Application using Streamlit with ngrok
# This creates a web-based dashboard accessible via a public URL in a new tab

# Install Streamlit and pyngrok
!pip install streamlit plotly openpyxl pyngrok --quiet

# Set environment variables for the dashboard script
import os
os.environ['DB_PATH'] = DB_PATH
os.environ['PROJECT_PATH'] = PROJECT_PATH

# Import required modules
import subprocess
import threading
import time
from IPython.display import HTML, display
from pyngrok import ngrok

# ============================================================================
# Ngrok authtoken (already configured)
# ============================================================================
# Set ngrok authtoken for public URL access
ngrok.set_auth_token("375PDlRat74qHXZgq7QqF4BMQPH_56MZXpBvE8VAg4iEfqcqR")
# ============================================================================

# Kill any existing Streamlit and ngrok processes
subprocess.run(['pkill', '-f', 'streamlit'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-f', 'ngrok'], stderr=subprocess.DEVNULL)

print("⏳ Starting Streamlit server...")

# Start Streamlit in the background
def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 
        f'{PROJECT_PATH}/dashboard_app.py',
        '--server.port=8501',
        '--server.address=127.0.0.1',  # Localhost only, ngrok will expose it
        '--server.headless=true',
        '--browser.gatherUsageStats=false'
    ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Wait for Streamlit to start
print("⏳ Waiting for Streamlit to start (15 seconds)...")
time.sleep(15)

# Verify Streamlit is running
result = subprocess.run(['pgrep', '-f', 'streamlit'], capture_output=True, text=True)
if result.returncode == 0:
    print("✅ Streamlit is running!")
else:
    print("⚠️ Streamlit may not have started. Check for errors above.")

# Create ngrok tunnel
print("\n🌐 Creating public URL with ngrok...")
try:
    # Open ngrok tunnel to port 8501
    public_url = ngrok.connect(8501)
    
    print(f"\n✅ Dashboard is ready!")
    print(f"🌐 Public URL: {public_url}")
    print(f"\n📱 Click the button below to open in a new tab:")
    
    # Create clickable link
    display(HTML(f'''
    <div style="text-align: center; margin: 20px 0; padding: 20px; background: #f0f0f0; border-radius: 8px;">
        <h3 style="margin-top: 0;">🚀 Open Dashboard in New Tab</h3>
        <a href="{public_url}" target="_blank" 
           style="font-size: 24px; padding: 15px 30px; background: #667eea; 
                  color: white; text-decoration: none; border-radius: 8px; 
                  display: inline-block; font-weight: bold; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
            Open Dashboard
        </a>
        <p style="margin-top: 15px; color: #666; font-size: 14px;">
            💡 This link opens in a new tab/window
        </p>
    </div>
    '''))
    
    print(f"\n💡 The dashboard will run until you interrupt this cell.")
    print(f"🛑 To stop: Runtime → Interrupt execution (or restart runtime)")
    print(f"\n📌 Your dashboard URL: {public_url}")
    
except Exception as e:
    error_msg = str(e)
    if "authtoken" in error_msg.lower() or "4018" in error_msg:
        print("\n❌ Ngrok authentication required!")
        print("\n📝 To fix this:")
        print("   1. Sign up for a free ngrok account: https://dashboard.ngrok.com/signup")
        print("   2. Get your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken")
        print("   3. Uncomment and update the ngrok.set_auth_token() line above")
        print("   4. Re-run this cell")
        print("\n💡 It's free and only takes a minute!")
    else:
        print(f"\n⚠️ Ngrok error: {e}")
        print("\n📌 Alternative: Use the 'Interactive Reports' cell (Cell 7) which runs in the notebook.")


## Interactive Reports with Charts and GUI

This cell creates a nice interactive interface with formatted reports and charts.

In [ ]:
# Interactive Reports with Charts and Nice Formatting
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display
import ipywidgets as widgets
from datetime import datetime, date
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import io
from pathlib import Path

# Install required packages if needed
try:
    import plotly
except ImportError:
    print("Installing plotly...")
    !pip install plotly kaleido openpyxl
    import plotly.graph_objects as go
    import plotly.express as px

try:
    import ipywidgets
except ImportError:
    print("Installing ipywidgets...")
    !pip install ipywidgets
    import ipywidgets

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Make sure we have the database connection
# (This assumes db is already created from a previous cell, or create it here)
if 'db' not in locals():
    import sys
    import os
    if PROJECT_PATH not in sys.path:
        sys.path.insert(0, PROJECT_PATH)
    from database.db_connection import create_db_session
    db = create_db_session(DB_PATH)

# Import query functions
from reports.contract_queries import get_all_contracts, get_active_contracts, get_contracts_by_commodity
from reports.settlement_queries import get_all_settlements
from reports.bin_queries import get_all_bins

def create_formatted_report():
    """Create a nicely formatted HTML report."""
    
    # Get data
    contracts = get_all_contracts(db)
    settlements = get_all_settlements(db)
    bins = get_all_bins(db)
    
    # Build HTML report
    html = f"""
    <style>
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }}
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            border-radius: 10px;
            margin-bottom: 20px;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }}
        .header h1 {{
            margin: 0;
            font-size: 2.5em;
        }}
        .header p {{
            margin: 10px 0 0 0;
            opacity: 0.9;
        }}
        .section {{
            background: white;
            padding: 20px;
            margin: 20px 0;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        }}
        .section h2 {{
            color: #667eea;
            border-bottom: 3px solid #667eea;
            padding-bottom: 10px;
        }}
        .stat-box {{
            display: inline-block;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px 25px;
            margin: 10px;
            border-radius: 8px;
            text-align: center;
            min-width: 150px;
        }}
        .stat-box .number {{
            font-size: 2em;
            font-weight: bold;
        }}
        .stat-box .label {{
            font-size: 0.9em;
            opacity: 0.9;
        }}
        table {{
            width: 100%;
            border-collapse: collapse;
            margin: 20px 0;
        }}
        th {{
            background-color: #667eea;
            color: white;
            padding: 12px;
            text-align: left;
            font-weight: 600;
        }}
        td {{
            padding: 10px;
            border-bottom: 1px solid #ddd;
        }}
        tr:hover {{
            background-color: #f5f5f5;
        }}
        .status-active {{
            background-color: #4caf50;
            color: white;
            padding: 5px 10px;
            border-radius: 5px;
            font-size: 0.9em;
        }}
        .status-inactive {{
            background-color: #f44336;
            color: white;
            padding: 5px 10px;
            border-radius: 5px;
            font-size: 0.9em;
        }}
    </style>
    
    <div class="header">
        <h1>🌾 Bushel Management Report</h1>
        <p>Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}</p>
    </div>
    
    <div class="section">
        <h2>📊 Summary Statistics</h2>
        <div class="stat-box">
            <div class="number">{len(contracts)}</div>
            <div class="label">Total Contracts</div>
        </div>
        <div class="stat-box">
            <div class="number">{len([c for c in contracts if c.status == 'Active'])}</div>
            <div class="label">Active Contracts</div>
        </div>
        <div class="stat-box">
            <div class="number">{len(settlements)}</div>
            <div class="label">Settlements</div>
        </div>
        <div class="stat-box">
            <div class="number">{len(bins)}</div>
            <div class="label">Storage Bins</div>
        </div>
    </div>
    """
    
    # Add contracts table
    if contracts:
        html += """
        <div class="section">
            <h2>📋 Contracts</h2>
            <table>
                <thead>
                    <tr>
                        <th>Contract #</th>
                        <th>Commodity</th>
                        <th>Bushels</th>
                        <th>Price</th>
                        <th>Basis</th>
                        <th>Status</th>
                        <th>Date Sold</th>
                        <th>Buyer</th>
                    </tr>
                </thead>
                <tbody>
        """
        
        for contract in contracts[:20]:  # Show first 20
            status_class = "status-active" if contract.status == "Active" else "status-inactive"
            html += f"""
                    <tr>
                        <td><strong>{contract.contract_number or 'N/A'}</strong></td>
                        <td>{contract.commodity or 'N/A'}</td>
                        <td>{contract.bushels or 0:,}</td>
                        <td>${contract.price or 0:.2f}</td>
                        <td>${contract.basis or 0:.2f}</td>
                        <td><span class="{status_class}">{contract.status or 'N/A'}</span></td>
                        <td>{contract.date_sold or 'N/A'}</td>
                        <td>{contract.buyer_name or 'N/A'}</td>
                    </tr>
            """
        
        html += """
                </tbody>
            </table>
        </div>
        """
    
    html += "</div>"
    
    return HTML(html)

def create_charts():
    """Create interactive charts."""
    
    # Get data
    contracts = get_all_contracts(db)
    settlements = get_all_settlements(db)
    bins = get_all_bins(db)
    
    # Create figure with subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Bushel Management Analytics Dashboard', fontsize=16, fontweight='bold')
    
    # Chart 1: Contracts by Commodity
    if contracts:
        df_contracts = pd.DataFrame([{
            'Commodity': c.commodity or 'Unknown',
            'Bushels': c.bushels or 0,
            'Status': c.status or 'Unknown'
        } for c in contracts])
        
        if not df_contracts.empty:
            commodity_totals = df_contracts.groupby('Commodity')['Bushels'].sum().sort_values(ascending=False)
            axes[0, 0].bar(commodity_totals.index, commodity_totals.values, color='#667eea')
            axes[0, 0].set_title('Total Bushels by Commodity', fontweight='bold')
            axes[0, 0].set_xlabel('Commodity')
            axes[0, 0].set_ylabel('Bushels')
            axes[0, 0].tick_params(axis='x', rotation=45)
            for i, v in enumerate(commodity_totals.values):
                axes[0, 0].text(i, v, f'{v:,.0f}', ha='center', va='bottom')
    
    # Chart 2: Contracts by Status
    if contracts:
        status_counts = df_contracts['Status'].value_counts()
        colors = ['#4caf50' if s == 'Active' else '#f44336' for s in status_counts.index]
        axes[0, 1].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', 
                       colors=colors, startangle=90)
        axes[0, 1].set_title('Contracts by Status', fontweight='bold')
    
    # Chart 3: Price Distribution
    if contracts:
        prices = [c.price for c in contracts if c.price]
        if prices:
            axes[1, 0].hist(prices, bins=20, color='#764ba2', edgecolor='black')
            axes[1, 0].set_title('Price Distribution', fontweight='bold')
            axes[1, 0].set_xlabel('Price per Bushel ($)')
            axes[1, 0].set_ylabel('Frequency')
            axes[1, 0].axvline(pd.Series(prices).mean(), color='red', linestyle='--', 
                              label=f'Mean: ${pd.Series(prices).mean():.2f}')
            axes[1, 0].legend()
    
    # Chart 4: Bin Capacity vs Current Quantity
    if bins:
        df_bins = pd.DataFrame([{
            'Bin': b.bin_number or 'Unknown',
            'Capacity': b.capacity_bushels or 0,
            'Current': b.current_quantity or 0
        } for b in bins])
        
        if not df_bins.empty and len(df_bins) > 0:
            x = range(len(df_bins))
            width = 0.35
            axes[1, 1].bar([i - width/2 for i in x], df_bins['Capacity'], width, 
                          label='Capacity', color='#667eea')
            axes[1, 1].bar([i + width/2 for i in x], df_bins['Current'], width, 
                          label='Current', color='#4caf50')
            axes[1, 1].set_title('Bin Capacity vs Current Quantity', fontweight='bold')
            axes[1, 1].set_xlabel('Bin')
            axes[1, 1].set_ylabel('Bushels')
            axes[1, 1].set_xticks(x)
            axes[1, 1].set_xticklabels(df_bins['Bin'], rotation=45, ha='right')
            axes[1, 1].legend()
    
    plt.tight_layout()
    plt.show()

# Create interactive widget for report selection
def create_interactive_dashboard():
    """Create an interactive dashboard with dropdown selection."""
    
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            output.clear_output()
            if b.description == "📊 Show Formatted Report":
                display(create_formatted_report())
            elif b.description == "📈 Show Charts":
                create_charts()
            elif b.description == "📋 Show Contracts Table":
                contracts = get_all_contracts(db)
                if contracts:
                    df = pd.DataFrame([{
                        'Contract #': c.contract_number,
                        'Commodity': c.commodity,
                        'Bushels': c.bushels,
                        'Price': f"${c.price:.2f}" if c.price else "N/A",
                        'Status': c.status,
                        'Date Sold': c.date_sold,
                        'Buyer': c.buyer_name
                    } for c in contracts])
                    display(df.style.background_gradient(subset=['Bushels']).format({'Bushels': '{:,.0f}'}))
                else:
                    print("No contracts found.")
    
    # Create buttons
    button1 = widgets.Button(description="📊 Show Formatted Report", 
                            button_style='info', layout=widgets.Layout(width='200px', height='40px'))
    button2 = widgets.Button(description="📈 Show Charts", 
                            button_style='success', layout=widgets.Layout(width='200px', height='40px'))
    button3 = widgets.Button(description="📋 Show Contracts Table", 
                            button_style='warning', layout=widgets.Layout(width='200px', height='40px'))
    
    button1.on_click(on_button_click)
    button2.on_click(on_button_click)
    button3.on_click(on_button_click)
    
    # Display buttons and output
    display(widgets.HBox([button1, button2, button3]))
    display(output)
    
    # Show initial report
    display(create_formatted_report())
    create_charts()

# Install ipywidgets if needed
try:
    import ipywidgets
except ImportError:
    print("Installing ipywidgets...")
    !pip install ipywidgets
    import ipywidgets

# Run the interactive dashboard
create_interactive_dashboard()

## Enhanced Interactive Dashboard

**This cell includes all the advanced features:**
- 🔍 **Dropdown filters** (Commodity, Status, Date Range)
- 📊 **Formatted HTML reports**
- 📈 **Matplotlib charts** (bar, pie, histogram, line)
- 📉 **Interactive Plotly charts** (zoom, pan, hover)
- 📥 **Export to Excel**
- 📄 **Export to PDF**

In [ ]:
# Enhanced Interactive Dashboard with Filters, Export, and Plotly Charts
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display
import ipywidgets as widgets
from datetime import datetime, date
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Install required packages
try:
    import plotly
except ImportError:
    print("Installing plotly...")
    !pip install plotly kaleido openpyxl reportlab
    import plotly.graph_objects as go
    import plotly.express as px

try:
    import ipywidgets
except ImportError:
    print("Installing ipywidgets...")
    !pip install ipywidgets
    import ipywidgets

# Set up plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Ensure database connection
if 'db' not in locals():
    import sys
    import os
    if PROJECT_PATH not in sys.path:
        sys.path.insert(0, PROJECT_PATH)
    from database.db_connection import create_db_session
    db = create_db_session(DB_PATH)

from reports.contract_queries import get_all_contracts, get_active_contracts
from reports.settlement_queries import get_all_settlements
from reports.bin_queries import get_all_bins

def create_formatted_report_with_data(contracts, settlements, bins):
    """Create formatted HTML report."""
    html = f"""
    <style>
        body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 20px; background-color: #f5f5f5; }}
        .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 30px; border-radius: 10px; margin-bottom: 20px; }}
        .header h1 {{ margin: 0; font-size: 2.5em; }}
        .section {{ background: white; padding: 20px; margin: 20px 0; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
        .section h2 {{ color: #667eea; border-bottom: 3px solid #667eea; padding-bottom: 10px; }}
        .stat-box {{ display: inline-block; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px 25px; margin: 10px; border-radius: 8px; text-align: center; min-width: 150px; }}
        .stat-box .number {{ font-size: 2em; font-weight: bold; }}
        table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
        th {{ background-color: #667eea; color: white; padding: 12px; text-align: left; }}
        td {{ padding: 10px; border-bottom: 1px solid #ddd; }}
        tr:hover {{ background-color: #f5f5f5; }}
        .status-active {{ background-color: #4caf50; color: white; padding: 5px 10px; border-radius: 5px; }}
    </style>
    <div class="header"><h1>🌾 Bushel Management Report</h1><p>Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}</p></div>
    <div class="section"><h2>📊 Summary</h2>
    <div class="stat-box"><div class="number">{len(contracts)}</div><div>Contracts</div></div>
    <div class="stat-box"><div class="number">{len(settlements)}</div><div>Settlements</div></div>
    <div class="stat-box"><div class="number">{len(bins)}</div><div>Bins</div></div>
    </div>
    """
    if contracts:
        html += "<div class=\"section\"><h2>📋 Contracts</h2><table><thead><tr><th>Contract #</th><th>Commodity</th><th>Bushels</th><th>Price</th><th>Basis</th><th>Status</th><th>Date Sold</th><th>Buyer</th></tr></thead><tbody>"
        for c in contracts[:50]:
            status_class = "status-active" if c.status == "Active" else ""
            html += f"<tr><td><strong>{c.contract_number or 'N/A'}</strong></td><td>{c.commodity or 'N/A'}</td><td>{c.bushels or 0:,}</td><td>${c.price or 0:.2f}</td><td>${c.basis or 0:.2f}</td><td><span class=\"{status_class}\">{c.status or 'N/A'}</span></td><td>{c.date_sold or 'N/A'}</td><td>{c.buyer_name or 'N/A'}</td></tr>"
        html += "</tbody></table></div>"
    return HTML(html)

def create_charts_with_data(contracts, settlements, bins):
    """Create matplotlib charts."""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Bushel Management Analytics', fontsize=16, fontweight='bold')
    if contracts:
        df = pd.DataFrame([{'Commodity': c.commodity or 'Unknown', 'Bushels': c.bushels or 0, 'Status': c.status or 'Unknown', 'Price': c.price or 0} for c in contracts])
        if not df.empty:
            totals = df.groupby('Commodity')['Bushels'].sum().sort_values(ascending=False)
            axes[0, 0].bar(totals.index, totals.values, color='#667eea')
            axes[0, 0].set_title('Bushels by Commodity', fontweight='bold')
            axes[0, 0].tick_params(axis='x', rotation=45)
            status_counts = df['Status'].value_counts()
            colors = ['#4caf50' if s == 'Active' else '#f44336' for s in status_counts.index]
            axes[0, 1].pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', colors=colors)
            axes[0, 1].set_title('Contracts by Status', fontweight='bold')
            prices = [p for p in df['Price'] if p and p > 0]
            if prices:
                axes[1, 0].hist(prices, bins=20, color='#764ba2', edgecolor='black')
                axes[1, 0].set_title('Price Distribution', fontweight='bold')
                axes[1, 0].axvline(pd.Series(prices).mean(), color='red', linestyle='--', label=f'Mean: ${pd.Series(prices).mean():.2f}')
                axes[1, 0].legend()
    plt.tight_layout()
    plt.show()

def create_plotly_charts(contracts, settlements, bins):
    """Create interactive Plotly charts."""
    if not contracts:
        display(HTML("<p>No contracts to display.</p>"))
        return
    df = pd.DataFrame([{
        'Contract': c.contract_number or 'N/A',
        'Commodity': c.commodity or 'Unknown',
        'Bushels': c.bushels or 0,
        'Price': c.price or 0,
        'Basis': c.basis or 0,
        'Status': c.status or 'Unknown',
        'Date Sold': c.date_sold
    } for c in contracts])
    fig = make_subplots(rows=2, cols=2, subplot_titles=('Bushels by Commodity', 'Price vs Bushels (Scatter)', 'Contracts Over Time', 'Status Distribution'),
                       specs=[[{"type": "bar"}, {"type": "scatter"}], [{"type": "scatter"}, {"type": "pie"}]])
    if not df.empty:
        commodity_totals = df.groupby('Commodity')['Bushels'].sum().sort_values(ascending=False)
        fig.add_trace(go.Bar(x=commodity_totals.index, y=commodity_totals.values, marker_color='#667eea', name='Bushels'), row=1, col=1)
        fig.add_trace(go.Scatter(x=df['Bushels'], y=df['Price'], mode='markers', marker=dict(size=10, color='#764ba2'), text=df['Contract'], hovertemplate='Contract: %{text}<br>Bushels: %{x}<br>Price: $%{y}<extra></extra>'), row=1, col=2)
        if 'Date Sold' in df.columns and df['Date Sold'].notna().any():
            df['Date Sold'] = pd.to_datetime(df['Date Sold'])
            daily = df.groupby(df['Date Sold'].dt.date).size()
            fig.add_trace(go.Scatter(x=daily.index, y=daily.values, mode='lines+markers', line=dict(color='#4caf50', width=3), fill='tozeroy'), row=2, col=1)
        status_counts = df['Status'].value_counts()
        fig.add_trace(go.Pie(labels=status_counts.index, values=status_counts.values, marker_colors=['#4caf50', '#f44336', '#ff9800']), row=2, col=2)
    fig.update_layout(height=800, showlegend=False, title_text="Interactive Bushel Management Dashboard", title_x=0.5)
    fig.show()
    # 3D Scatter plot
    if len(df) > 0 and df['Basis'].notna().any():
        fig_3d = go.Figure(data=go.Scatter3d(x=df['Bushels'], y=df['Price'], z=df['Basis'], mode='markers', marker=dict(size=8, color=df['Price'], colorscale='Viridis'), text=df['Contract'], hovertemplate='Contract: %{text}<br>Bushels: %{x}<br>Price: $%{y}<br>Basis: $%{z}<extra></extra>'))
        fig_3d.update_layout(title='3D View: Bushels vs Price vs Basis', scene=dict(xaxis_title='Bushels', yaxis_title='Price ($)', zaxis_title='Basis ($)'))
        fig_3d.show()

def export_to_excel(contracts, settlements, bins):
    """Export to Excel."""
    try:
        from openpyxl import Workbook
        from openpyxl.styles import Font, PatternFill, Alignment
        wb = Workbook()
        ws = wb.active
        ws.title = "Contracts"
        headers = ['Contract #', 'Commodity', 'Bushels', 'Price', 'Basis', 'Status', 'Date Sold', 'Buyer']
        ws.append(headers)
        header_fill = PatternFill(start_color="667eea", end_color="667eea", fill_type="solid")
        header_font = Font(bold=True, color="FFFFFF")
        for cell in ws[1]:
            cell.fill = header_fill
            cell.font = header_font
            cell.alignment = Alignment(horizontal='center')
        for c in contracts:
            ws.append([c.contract_number, c.commodity, c.bushels, c.price, c.basis, c.status, c.date_sold, c.buyer_name])
        if settlements:
            ws2 = wb.create_sheet("Settlements")
            ws2.append(['Settlement ID', 'Contract', 'Commodity', 'Bushels', 'Price', 'Date Delivered'])
            for s in settlements:
                ws2.append([s.settlement_ID, s.contract_id, s.commodity, s.bushels, s.price, s.date_delivered])
        output_path = f"{PROJECT_PATH}/bushel_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
        wb.save(output_path)
        display(HTML(f"<div style='background: #4caf50; color: white; padding: 15px; border-radius: 5px;'>✓ Excel exported: {output_path}</div>"))
    except Exception as e:
        display(HTML(f"<div style='background: #f44336; color: white; padding: 15px;'>✗ Error: {str(e)}</div>"))

def export_to_pdf(contracts, settlements, bins):
    """Export to PDF."""
    try:
        from reportlab.lib import colors
        from reportlab.lib.pagesizes import letter
        from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
        from reportlab.lib.styles import getSampleStyleSheet
        output_path = f"{PROJECT_PATH}/bushel_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
        doc = SimpleDocTemplate(output_path, pagesize=letter)
        story = []
        styles = getSampleStyleSheet()
        story.append(Paragraph("Bushel Management Report", styles['Title']))
        story.append(Spacer(1, 12))
        story.append(Paragraph(f"Generated: {datetime.now().strftime('%B %d, %Y at %I:%M %p')}", styles['Normal']))
        story.append(Spacer(1, 20))
        story.append(Paragraph(f"Total Contracts: {len(contracts)}", styles['Heading2']))
        story.append(Spacer(1, 12))
        if contracts:
            data = [['Contract #', 'Commodity', 'Bushels', 'Price', 'Status']]
            for c in contracts[:50]:
                data.append([c.contract_number or 'N/A', c.commodity or 'N/A', f"{c.bushels or 0:,}", f"${c.price or 0:.2f}", c.status or 'N/A'])
            table = Table(data)
            table.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey), ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke), ('ALIGN', (0, 0), (-1, -1), 'CENTER'), ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'), ('FONTSIZE', (0, 0), (-1, 0), 12), ('BOTTOMPADDING', (0, 0), (-1, 0), 12), ('BACKGROUND', (0, 1), (-1, -1), colors.beige), ('GRID', (0, 0), (-1, -1), 1, colors.black)]))
            story.append(table)
        doc.build(story)
        display(HTML(f"<div style='background: #4caf50; color: white; padding: 15px; border-radius: 5px;'>✓ PDF exported: {output_path}</div>"))
    except ImportError:
        !pip install reportlab
        export_to_pdf(contracts, settlements, bins)
    except Exception as e:
        display(HTML(f"<div style='background: #f44336; color: white; padding: 15px;'>✗ Error: {str(e)}</div>"))

# Create Enhanced Dashboard
def create_enhanced_dashboard():
    """Enhanced dashboard with filters, export, and Plotly charts."""
    all_contracts = get_all_contracts(db)
    all_settlements = get_all_settlements(db)
    all_bins = get_all_bins(db)
    output = widgets.Output(layout=widgets.Layout(height='600px', overflow='auto'))
    
    # Create filters
    commodities = ['All'] + sorted(list(set([c.commodity for c in all_contracts if c.commodity])))
    statuses = ['All'] + sorted(list(set([c.status for c in all_contracts if c.status])))
    
    commodity_filter = widgets.Dropdown(options=commodities, value='All', description='Commodity:', layout=widgets.Layout(width='200px'))
    status_filter = widgets.Dropdown(options=statuses, value='All', description='Status:', layout=widgets.Layout(width='200px'))
    date_from = widgets.DatePicker(description='From Date:', layout=widgets.Layout(width='200px'))
    date_to = widgets.DatePicker(description='To Date:', layout=widgets.Layout(width='200px'))
    
    def filter_contracts():
        filtered = all_contracts.copy()
        if commodity_filter.value != 'All':
            filtered = [c for c in filtered if c.commodity == commodity_filter.value]
        if status_filter.value != 'All':
            filtered = [c for c in filtered if c.status == status_filter.value]
        if date_from.value:
            filtered = [c for c in filtered if c.date_sold and c.date_sold >= date_from.value]
        if date_to.value:
            filtered = [c for c in filtered if c.date_sold and c.date_sold <= date_to.value]
        return filtered
    
    def on_show_report(b):
        with output:
            output.clear_output()
            display(create_formatted_report_with_data(filter_contracts(), all_settlements, all_bins))
    
    def on_show_charts(b):
        with output:
            output.clear_output()
            create_charts_with_data(filter_contracts(), all_settlements, all_bins)
    
    def on_show_plotly(b):
        with output:
            output.clear_output()
            create_plotly_charts(filter_contracts(), all_settlements, all_bins)
    
    def on_export_excel(b):
        with output:
            output.clear_output()
            export_to_excel(filter_contracts(), all_settlements, all_bins)
    
    def on_export_pdf(b):
        with output:
            output.clear_output()
            export_to_pdf(filter_contracts(), all_settlements, all_bins)
    
    # Create buttons
    show_report_btn = widgets.Button(description="📊 Show Report", button_style='info', layout=widgets.Layout(width='180px', height='40px'))
    show_charts_btn = widgets.Button(description="📈 Matplotlib", button_style='success', layout=widgets.Layout(width='180px', height='40px'))
    show_plotly_btn = widgets.Button(description="📉 Plotly Charts", button_style='primary', layout=widgets.Layout(width='180px', height='40px'))
    export_excel_btn = widgets.Button(description="📥 Export Excel", button_style='warning', layout=widgets.Layout(width='180px', height='40px'))
    export_pdf_btn = widgets.Button(description="📄 Export PDF", button_style='danger', layout=widgets.Layout(width='180px', height='40px'))
    
    show_report_btn.on_click(on_show_report)
    show_charts_btn.on_click(on_show_charts)
    show_plotly_btn.on_click(on_show_plotly)
    export_excel_btn.on_click(on_export_excel)
    export_pdf_btn.on_click(on_export_pdf)
    
    # Auto-update on filter change
    def on_filter_change(change):
        with output:
            output.clear_output()
            display(HTML(f"<h3>🔍 Filtered Results: {len(filter_contracts())} contracts match your criteria</h3>"))
    
    commodity_filter.observe(on_filter_change, names='value')
    status_filter.observe(on_filter_change, names='value')
    
    # Display everything
    display(widgets.VBox([
        widgets.HTML("<h2>🌾 Enhanced Bushel Management Dashboard</h2>"),
        widgets.HTML("<h3>🔍 Filters</h3>"),
        widgets.HBox([commodity_filter, status_filter, date_from, date_to]),
        widgets.HTML("<h3>📊 Actions</h3>"),
        widgets.HBox([show_report_btn, show_charts_btn, show_plotly_btn]),
        widgets.HBox([export_excel_btn, export_pdf_btn]),
        output
    ]))
    
    # Show initial report and Plotly charts
    with output:
        display(create_formatted_report_with_data(filter_contracts(), all_settlements, all_bins))
        create_plotly_charts(filter_contracts(), all_settlements, all_bins)

# Run the enhanced dashboard
create_enhanced_dashboard()